In [ ]:
%matplotlib inline


# 2D Poisson Equation with Zero Neumann Condition (Cell Centered Formulation)

Here we use the *discretize* package to approximate the solution to a 2D
Poisson equation with zero Neumann boundary conditions.
For our tutorial, we consider the free-space electrostatic problem
for 2 electric point charges of opposite sign.
This tutorial focusses on:

    - approximating basic types of inner products
    - imposing the boundary condition by approximating a surface integral
    - basic disretization of point sources


## 1. Formulating the Problem

For this tutorial, we would like to compute the electric potential ($\phi$)
and electric fields ($\mathbf{e}$) in 2D that result from
a distribution of point charges in a vacuum. Given the electric permittiviy
is uniform within the domain and equal to the permittivity of free space, the physics
are defined by a 2D Poisson equation:

\begin{align}\nabla^2 \phi = - \frac{\rho}{\epsilon_0}\end{align}

where $\rho$ is the charge density and $\epsilon_0$ is
the permittivity of free space. A more practical
formulation for applying the finite volume method is to 
define the problem as two fist-order differential equations.
Starting with Gauss's law and Faraday's law in the case of
electrostatics (:cite:`griffiths1999`):

\begin{align}&\nabla \cdot \vec{e} = \frac{\rho}{\epsilon_0} \\
    &\nabla \times \vec{e} = \boldsymbol{0} \;\;\; \Rightarrow \;\;\; \vec{e} = -\nabla \phi \\
    &\textrm{s.t.} \;\;\; \frac{\partial \phi}{\partial n} \, \Big |_{\partial \Omega} = - \hat{n} \cdot \vec{e} \, \Big |_{\partial \Omega} = 0\end{align}

where the Neumann boundary condition on $\phi$ implies no electric flux leaves the system.
For 2 point charges of equal and opposite sign, the charge density is given by:

\begin{align}\rho = \rho_0 \big [ \delta ( \boldsymbol{r_+}) - \delta (\boldsymbol{r_-} ) \big ]
    :label: tutorial_eq_1_2_1\end{align}

where $\rho_0$ is a constant.
We chose this charge distribution so that we would not violate the boundary conditions.
Since the total electric charge is zero, the net flux leaving the system is zero.




## 2. Taking Inner Products

To solve this problem numerically, we take the inner product
of each differential equation with an appropriate test function.
Where $\psi$ is a scalar test function and $\vec{u}$ is a
vector test function:

\begin{align}\int_\Omega \psi (\nabla \cdot \vec{e}) \, dv = \frac{1}{\epsilon_0} \int_\Omega \psi \rho \, dv
    :label: tutorial_eq_1_2_3\end{align}

and

\begin{align}\int_\Omega \vec{u} \cdot \vec{e} \, dv = - \int_\Omega \vec{u} \cdot (\nabla \phi ) \, dv
    :label: tutorial_eq_1_2_4\end{align}

In the case of Gauss' law, we have a volume integral containing the Dirac delta function.
Thus expression :eq:`tutorial_eq_1_2_3` becomes:

\begin{align}\int_\Omega \psi (\nabla \cdot \vec{e}) \, dv = \frac{1}{\epsilon_0} \psi \, q
    :label: tutorial_eq_1_2_5\end{align}

where $q$ represents an integrated charge density.




## 3. Discretizing the Inner Products

Here, we let $\boldsymbol{\phi}$ be the discrete representation
of the electic potential $\phi$ at cell centers. Since $\vec{e} = -\nabla \phi$,
it is natural for discrete representation of the electric field $\boldsymbol{e}$
to live on the faces; implying the discrete divergence operator maps from faces to cell centers.

In tutorials for inner products, we showed how to approximate most classes
of inner products. Since the numerical divergence of $\boldsymbol{e}$ is
mapped to cell centers, the discrete representation of the test function $\boldsymbol{\psi}$
and the integrated charge density $\boldsymbol{q}$ must live at cell centers.

Approximating the inner products in expression :eq:`tutorial_eq_1_2_5`
according to the finite volume method, we obtain:

\begin{align}\boldsymbol{\psi^T M_c D e} = \frac{1}{\epsilon_0} \boldsymbol{\psi^T q}
    :label: tutorial_eq_1_2_10\end{align}

where

    - $\boldsymbol{M_c}$ is the inner product matrix at cell centers,
    - $\boldsymbol{D}$ is the discrete divergence operator
    - $\boldsymbol{q}$ is a discrete representation for the integrated charge density for each cell

The easiest way to discretize the source is to let $\boldsymbol{q_i}=\rho_0$ at the nearest
cell center to the positive charge and to let $\boldsymbol{q_i}=-\rho_0$ at
the nearest cell center to the negative charge. The value is zero for all other cells.

Now we approximate the inner product in expression :eq:`tutorial_eq_1_2_4`.
Since $\boldsymbol{\phi}$ lives at cell centers, it would be natural for the
discrete gradient operator to map from centers to faces. Unfortunately for 
boundary faces, this would require knowledge of the electric potential at cell
centers outside our domain.
For the right-hand side, we must use the identity
$\vec{u} \cdot \nabla \phi = \nabla \cdot \phi\vec{u} - \phi \nabla \cdot \vec{u}$
and apply the divergence theorem such that expression :eq:`tutorial_eq_1_2_4` becomes:

\begin{align}\int_\Omega \vec{u} \cdot \vec{e} \, dv = \int_\Omega \phi \nabla \cdot \vec{u} \, dv - \oint_{\partial \Omega} \phi \hat{n} \cdot \vec{u} \, da
    :label: tutorial_eq_1_2_11\end{align}

According to expression :eq:`tutorial_eq_1_2_1`,
$- \frac{\partial \phi}{\partial n} = 0$ on the boundaries.
To accurately compute the electric potentials at cell centers,
we must implement the boundary conditions by approximating the surface integral.
In this case, expression :eq:`tutorial_eq_1_2_11` is approximated by:

\begin{align}\boldsymbol{u^T M_f \, e} = \boldsymbol{u^T D^T M_c \, \phi} - \boldsymbol{u^T B \, \phi} = - \boldsymbol{\tilde{G} \, \phi}
    :label: tutorial_eq_1_2_12\end{align}

where

    - $\boldsymbol{M_c}$ is the inner product matrix at cell centers
    - $\boldsymbol{M_f}$ is the inner product matrix at faces
    - $\boldsymbol{D}$ is the discrete divergence operator
    - $\boldsymbol{B}$ is a sparse matrix that imposes the Neumann boundary condition
    - $\boldsymbol{\tilde{G}} = - \boldsymbol{D^T M_c} + \boldsymbol{B}$ acts as a modified gradient operator with boundary conditions included




## 4. Solvable Linear System

By combining the discrete representations from expressions
:eq:`tutorial_eq_1_2_10` and :eq:`tutorial_eq_1_2_12` and factoring like-terms
we obtain:

\begin{align}- \boldsymbol{M_c D M_f^{-1} \tilde{G} \, \phi} = \frac{1}{\epsilon_0} \boldsymbol{q}
    :label: tutorial_eq_1_2_13\end{align}

Once the electric potential at cell centers has been computed, the electric field on
the faces can be computed using expression :eq:`tutorial_eq_1_2_12`:

\begin{align}\boldsymbol{e} = - \boldsymbol{M_f^{-1} \tilde{G} \, \phi}\end{align}




## 5. Implement Discretize



Import the necessary packages for the tutorial.



In [ ]:
from discretize import TensorMesh
from pymatsolver import SolverLU
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from discretize.utils import sdiag

mpl.rcParams.update({'font.size':14})

Construct a mesh.



In [ ]:
h = 2*np.ones(51)
mesh = TensorMesh([h, h], "CC")

Construct the necessary discrete operators and inner product matrices.



In [ ]:
DIV = mesh.faceDiv                                        # discrete divergence operator
Mc = sdiag(mesh.vol)                                      # cell center inner product matrix
Mf_inv = mesh.get_face_inner_product(invert_matrix=True)  # inverse of face inner product matrix

mesh.set_cell_gradient_BC(['neumann','neumann'])  # Set zero Neumann condition on gradient
G = mesh.cell_gradient                            # Modified gradient operator G = -D^T * Mc + B

Form the linear system of equations to be solved.



In [ ]:
A = - Mc * DIV * Mf_inv * G

Construct the right-hand side.



In [ ]:
xycc = mesh.gridCC
kneg = (xycc[:, 0] == -10) & (xycc[:, 1] == 0)      # -ve charge at (-10, 0)
kpos = (xycc[:, 0] == 10) & (xycc[:, 1] == 0)       # +ve charge at (10, 0)

rho = np.zeros(mesh.nC)
rho[kneg] = -1
rho[kpos] = 1

Solve for the electric potential at cell centers and the electric field
on the faces.



In [ ]:
AinvM = SolverLU(A)     # Define the inverse of A using direct solver
phi = AinvM * rho       # Compute electric potential at cell centers
E = - Mf_inv * G * phi  # Compute electric field on faces

Plot the source term, electric potential and electric field.



In [ ]:
fig = plt.figure(figsize=(12, 4))

ax1 = fig.add_subplot(131)
mesh.plotImage(rho, v_type="CC", ax=ax1)
ax1.set_title("Charge Density")

ax2 = fig.add_subplot(132)
mesh.plotImage(phi, v_type="CC", ax=ax2)
ax2.set_title("Electric Potential")

ax3 = fig.add_subplot(133)
mesh.plotImage(
    E, ax=ax3, v_type="F", view="vec", stream_opts={"color": "w", "density": 1.0}
)
ax3.set_title("Electric Fields")

plt.tight_layout()